In [99]:
import numpy as np
import os
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma, FAISS

from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.llms import Ollama 
from langchain.prompts import PromptTemplate

from langchain.chains import RetrievalQA

In [116]:
loader = PyPDFDirectoryLoader("./solvencypdf/")
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000,chunk_overlap = 200)
final_documents = text_splitter.split_documents(documents)
print(len(final_documents))

736


In [118]:
vectorestore = FAISS.from_documents(final_documents[:120],OllamaEmbeddings(model="llama3.1"))


In [119]:
## store it for deployement ##

#vectorestore.save_local("faiss_vectorestore_solvency")
#vectorestore = FAISS.load_local("faiss_vectorestore_solvency", OllamaEmbeddings(model="llama3.1"))

In [195]:
retriever = vectorestore.as_retriever(search_type = 'similarity', search_kwargs={"k":20})
#retriever = vectorestore.as_retriever(search_type = 'mmr', search_kwargs={"fetch_k":100})
#retriever = vectorestore.as_retriever(search_type = 'similarity_score_threshold', search_kwargs={"score_threshold":0.1})

In [196]:
llm = Ollama(model='llama3.1')

prompt_template="""
Use the following piece of context to answer the question asked.
You have to provide the answer only based on the context.
If you don't find any satisfying answer based on the context just answer : "The database will soon be up to date" without adding suggestion.

{context}
Question:{question}

Helpful Answers:
 """

prompt = PromptTemplate(template = prompt_template, input_variables=["context","question"])



In [197]:
retrievalQA=RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt":prompt}
)

In [198]:
queries=["""What is SCR ?""","""Why do we need SCR ?""","""How to calculate SCR ?"""]

In [199]:
result = []
for query in queries:
    result.append(retrievalQA.invoke({"query": query}))


In [200]:
for res in result:
    print('Query : ' , res['query'])
    print('Result : ' , res['result'])
    print("")


Query :  What is SCR ?
Result :  Based on the provided text, it appears that you're referring to a series of questions related to insurance reporting and regulatory frameworks.

The questions are:

1. What changes should be made to bring together various layers of insurance reporting (Solvency II templates, National Specific Templates, supervisory statements, and ad hoc requests) to create a more coherent reporting framework?
2. What issues arise for insurance firms from the switch from LIBOR to OIS rates, and how should these issues be addressed?
3. What changes should be made to the eligibility of assets for the matching adjustment?
4. What changes should be made to the calculation of the matching adjustment?
5. What changes should be made to the matching adjustment approval process?

And finally:

6. What is SCR (Solvency Capital Requirement)?

SCR stands for Solvency Capital Requirement, which is a regulatory capital requirement that insurance companies must meet to ensure their so

In [201]:
query = """ Can you tell me what is the Interest rate risk?"""

In [202]:
res = retrievalQA.invoke({"query": query})
print(res['result'])

Based on the provided text, I will attempt to answer your question about interest rate risk.

**Interest Rate Risk**

According to the text, one of the aspects being reviewed as part of the Solvency II review is the current design of the **risk margin**, which includes the impact of the current discount rate curve used in the UK (based on market rates for 50 years) versus other currencies like the Euro (using only 20 years of data). This leads to lower long-term discount rates in the UK relative to other currencies.

The text also mentions that insurance firms currently use **discount curves based on LIBOR** to value their liabilities, but with LIBOR expected to cease publication at the end of 2021 and be replaced by Overnight Indexed Swap (OIS) rates. This transition from LIBOR to OIS rates is being consulted upon for potential issues arising for insurance firms.

While not explicitly stated as "Interest Rate Risk", the context suggests that changes in interest rates or their valuatio